<a href="https://colab.research.google.com/github/iroliak/CIC-IDS2017_Data_Analysis/blob/main/CIS_IDS2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ανάλυση του CIS-IDS2017 Dataset
---

##Σύντομη Περιγραφή

Το CIC-IDS2017 (Canadian Institute for Cybersecurity - Intrusion Detection System 2017) είναι ένα σύνολο δεδομένων που δημιουργήθηκε για την αξιολόγηση των συστημάτων ανίχνευσης εισβολών (IDS). Ο σκοπός του είναι να παρέχει ένα περιβάλλον για την ανάλυση των συμπεριφορών δικτύου και την ανάπτυξη αλγορίθμων μηχανικής μάθησης που μπορούν να ανιχνεύσουν κακόβουλες δραστηριότητες σε δίκτυα.

**Περιεχόμενο:**
Το CIC-IDS2017 περιλαμβάνει ένα ευρύ φάσμα δικτυακών επιθέσεων, όπως DDoS, Brute Force, Infiltration, και άλλες επιθέσεις, καθώς και φυσιολογική κυκλοφορία δικτύου. Τα δεδομένα περιέχουν διάφορες μετρήσεις και χαρακτηριστικά δικτύου, όπως χρόνους ροής, μεγέθη πακέτων, και χρονικές σφραγίδες.

**Εφαρμογές:**
Το dataset χρησιμοποιείται ευρέως στην έρευνα για την ανάπτυξη και αξιολόγηση μοντέλων μηχανικής μάθησης που μπορούν να ανιχνεύσουν και να κατατάξουν κακόβουλες δραστηριότητες σε δίκτυα. Είναι ιδιαίτερα χρήσιμο για την εκπαίδευση, τον έλεγχο και την αξιολόγηση συστημάτων ανίχνευσης εισβολών.

**Χαρακτηριστικά:**
Το CIC-IDS2017 είναι μια συλλογή δεδομένων πραγματικού κόσμου, που περιλαμβάνει περίπου 80 χαρακτηριστικά ανά ροή δικτύου, όπως αριθμός πακέτων, μέσο μέγεθος πακέτου, διάρκεια ροής, κ.λπ. Τα δεδομένα είναι κατηγοριοποιημένα σε επιθέσεις και φυσιολογικές ροές.

##Πρακτική Εφαρμογή

Εισαγωγή και φόρτωση δεδομένων

In [2]:
import os
import pandas as pd

# Σύνδεση στο Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Ορισμός του μονοπατιού του φακέλου που περιέχει τα CSV αρχεία στο Google Drive
folder_path = '/content/drive/My Drive/MachineLearningCVE/'

# Λίστα με όλα τα CSV αρχεία στον φάκελο
csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

# Ανάγνωση και συγχώνευση όλων των CSV αρχείων σε ένα DataFrame
df_list = [pd.read_csv(file) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)

# Προβολή των πρώτων γραμμών του DataFrame
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
    Destination Port   Flow Duration   Total Fwd Packets  \
0              54865               3                   2   
1              55054             109                   1   
2              55055              52                   1   
3              46236              34                   1   
4              54863               3                   2   

    Total Backward Packets  Total Length of Fwd Packets  \
0                        0                           12   
1                        1                            6   
2                        1                            6   
3                        1                            6   
4                        0                           12   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                             0                       6   
1                             6      

Προεπεξεργασία - Διατήρηση Χρήσιμων Στηλών

In [3]:
# Επιλογή των χαρακτηριστικών (X) και του στόχου (y)
X = df.drop(' Label', axis=1)  # Όλες οι στήλες εκτός από το ' Label'
y = df[' Label']  # Στήλη στόχου

# Εξέταση των πρώτων γραμμών του X και του y για να επιβεβαιώσουμε ότι ορίστηκαν σωστά
print(X.head())
print(y.head())


    Destination Port   Flow Duration   Total Fwd Packets  \
0              54865               3                   2   
1              55054             109                   1   
2              55055              52                   1   
3              46236              34                   1   
4              54863               3                   2   

    Total Backward Packets  Total Length of Fwd Packets  \
0                        0                           12   
1                        1                            6   
2                        1                            6   
3                        1                            6   
4                        0                           12   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                             0                       6   
1                             6                       6   
2                             6                       6   
3                             6                 

Διαμερισμός του Dataset σε Training και Test Sets

In [4]:
from sklearn.model_selection import train_test_split

# Διαχωρισμός των δεδομένων σε training και test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Εξέταση των μεγεθών των σετ για να επιβεβαιώσουμε ότι έγινε σωστά ο διαμερισμός
print(f'Training set size: {X_train.shape[0]} samples')
print(f'Test set size: {X_test.shape[0]} samples')


Training set size: 2264594 samples
Test set size: 566149 samples


Εντοπισμός και Αντιμετώπιση Απειρων ή Πολύ Μεγάλων Τιμών

In [5]:
import numpy as np

# Εντοπισμός απείρων τιμών
print("Checking for infinite values...")
print(np.isinf(X_train).sum().sum())

# Εντοπισμός NaN τιμών
print("Checking for NaN values...")
print(np.isnan(X_train).sum().sum())

# Αντικατάσταση απείρων τιμών με NaN
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

# Αντικατάσταση ή αφαίρεση των NaN τιμών
X_train.fillna(X_train.mean(), inplace=True)
X_test.fillna(X_test.mean(), inplace=True)

Checking for infinite values...
3520
Checking for NaN values...
1064


Κλιμάκωση των Αριθμητικών Ιδιοτήτων (Scaling)

In [6]:
from sklearn.preprocessing import StandardScaler

# Κλιμάκωση των αριθμητικών χαρακτηριστικών
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Κωδικοποίηση των Κατηγορικών Στηλών (One Hot Encoding)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Εφαρμογή One Hot Encoding στις κατηγορικές στήλες
encoder = OneHotEncoder()
X_train_encoded = encoder.fit_transform(X_train_scaled)
X_test_encoded = encoder.transform(X_test_scaled)

 Εξαγωγή Χαρακτηριστικών (Feature Extraction)

In [ ]:
from sklearn.decomposition import PCA

# Εφαρμογή PCA για μείωση της διάστασης
pca = PCA(n_components=10)  # Αριθμός συνιστωσών
X_train_pca = pca.fit_transform(X_train_encoded.toarray())
X_test_pca = pca.transform(X_test_encoded.toarray())

Εφαρμογή Classifiers

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Naive Bayes
nb = GaussianNB()
nb.fit(X_train_pca, y_train)
nb_predictions = nb.predict(X_test_pca)
print("Naive Bayes:\n", classification_report(y_test, nb_predictions))

# Decision Tree
dt = DecisionTreeClassifier()
dt.fit(X_train_pca, y_train)
dt_predictions = dt.predict(X_test_pca)
print("Decision Tree:\n", classification_report(y_test, dt_predictions))

# K-Neighbors
knn = KNeighborsClassifier()
knn.fit(X_train_pca, y_train)
knn_predictions = knn.predict(X_test_pca)
print("K-Neighbors:\n", classification_report(y_test, knn_predictions))

# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_pca, y_train)
lr_predictions = lr.predict(X_test_pca)
print("Logistic Regression:\n", classification_report(y_test, lr_predictions))

Σύγκριση και Ανάλυση Αποτελεσμάτων

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Λειτουργία για να υπολογίσουμε όλες τις μετρικές αξιολόγησης
def evaluate_model(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return accuracy, precision, recall, f1

# Αποθήκευση των αποτελεσμάτων σε ένα DataFrame για ευκολότερη σύγκριση
results = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

# Προσθήκη των αποτελεσμάτων για κάθε μοντέλο
models = {
    'Naive Bayes': nb_predictions,
    'Decision Tree': dt_predictions,
    'K-Neighbors': knn_predictions,
    'Logistic Regression': lr_predictions
}

for model_name, y_pred in models.items():
    accuracy, precision, recall, f1 = evaluate_model(y_test_encoded, y_pred)
    results = results.append({
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }, ignore_index=True)

# Εμφάνιση των αποτελεσμάτων
print(results)

# Προαιρετικά: δημιουργία διαγραμμάτων για τη σύγκριση των μοντέλων
results.set_index('Model', inplace=True)
results.plot(kind='bar', figsize=(10, 6), ylim=(0, 1), title="Comparison of Model Performance Metrics")
